In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pymongo
import time

In [2]:
#Setting login values for web scrapping and chromedriver path
email = ''
password = ''
chrome_driver = ''
file = open('variables.txt', 'r') 
lines = file.readlines()
email = lines[0].split()[1].strip()
pwd = lines[1].split()[1].strip()
chrome_driver = lines[2].split()[1].strip()


In [3]:
#Launch LinkedIn using chrome driver
driver = webdriver.Chrome(chrome_driver)
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
time.sleep(5)
username_field = driver.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[1]/input')
time.sleep(1)
username_field.send_keys(email)
password = driver.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[2]/input')
time.sleep(1)
password.send_keys(pwd)
try:
    sign_in_button = driver.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[4]/button')
    sign_in_button.click()
except:
    sign_in_button = driver.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[3]/button')
    sign_in_button.click()
time.sleep(1)

In [4]:
#Setting up Mongo db

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["LinkedIn_Data"]
col = db["Company_content"]


In [5]:
#Code to search for contents for a spectific company
company_search = 'Oracle'

content_input = driver.find_element_by_xpath('/html/body/header/div[1]/form/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/input')
time.sleep(5)
content_input.send_keys(company_search)
content_input.send_keys(Keys.ENTER)
time.sleep(5)
try:
    content = driver.find_element_by_xpath('/html/body/div[7]/div[3]/div/div[1]/header/div/div/div[1]/ul/li[3]/button/span')
    content.click()
    time.sleep(5)
    date_posted = driver.find_element_by_xpath('/html/body/div[7]/div[3]/div/div[1]/header/div/div/div[2]/div/div/div/ul/li[2]/form/button/span')
    date_posted.click()
    time.sleep(5)
    one_day = driver.find_element_by_xpath('/html/body/div[7]/div[3]/div/div[1]/header/div/div/div[2]/div/div/div/ul/li[2]/form/div/fieldset/div/ul/li[1]/label/p/span')
    one_day.click()
    time.sleep(5)
    apply_button =  driver.find_element_by_xpath('/html/body/div[7]/div[3]/div/div[1]/header/div/div/div[2]/div/div/div/ul/li[2]/form/div/fieldset/div/div[1]/div[1]/button[2]/span')
    apply_button.click()
    time.sleep(5)
except:
    content = driver.find_element_by_xpath('/html/body/div[8]/div[3]/div/div[1]/header/div/div/div[1]/ul/li[3]/button/span')
    content.click()
    time.sleep(5)
    date_posted = driver.find_element_by_xpath('/html/body/div[8]/div[3]/div/div[1]/header/div/div/div[2]/div/div/div/ul/li[2]/form/button/span')
    date_posted.click()
    time.sleep(5)
    one_day = driver.find_element_by_xpath('/html/body/div[8]/div[3]/div/div[1]/header/div/div/div[2]/div/div/div/ul/li[2]/form/div/fieldset/div/ul/li[1]/label/p/span')
    one_day.click()
    time.sleep(5)
    apply_button =  driver.find_element_by_xpath('/html/body/div[8]/div[3]/div/div[1]/header/div/div/div[2]/div/div/div/ul/li[2]/form/div/fieldset/div/div[1]/div[1]/button[2]/span')
    apply_button.click()
    time.sleep(5)    




In [9]:
#Code to scrape web data
from bs4 import BeautifulSoup

def scroll(driver, timeout):
    scroll_pause_time = timeout
    data_dict={}

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    i=1

    while i<100:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height
        try:
            data_dict['id'] = i
            data_dict['data'] = driver.find_element_by_xpath('/html/body/div[7]/div[3]/div/div[2]/div/div[2]/div/ul/li['+str(i)+']/div/div[3]/div/div/span/span/span').text
            data_dict['User_name'] = driver.find_element_by_xpath('/html/body/div[7]/div[3]/div/div[2]/div/div[2]/div/ul/li['+str(i)+']/div/div[1]/a/div[3]/span[1]/span[1]/span').text
        

        except:
            print("Its a link")
            
        print(data_dict['User_name'])
        x = col.insert_one(data_dict)
        i = i+1

scroll(driver, 5)
   


Pravalika Dachepally
Sakshi Sachdev( Human resources)


DuplicateKeyError: E11000 duplicate key error collection: LinkedIn_Data.Company_content index: _id_ dup key: { _id: ObjectId('5ecd1c2bc9ce37f141462960') }

In [13]:
x = col.delete_many({})

In [14]:
find_uni = col.find({'User_name': {'$eq': 'Pravalika Dachepally'}})

In [15]:
print(find_uni)